<a href="https://colab.research.google.com/github/Spoomn/vision/blob/main/samples/notebooks/postgres_python_connector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/postgres_python_connector.ipynb)
# **Connect to Cloud SQL using the Cloud SQL Python Connector**

---

This notebook will be demonstrating how to connect and query data from a Cloud SQL database in an easy and efficient way all from within a jupyter style notebook! Let's have some fun!


### 📒 Using this interactive notebook

Click the **run** icons ▶️  of each section within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

## 🐍  **Cloud SQL Python Connector**
To connect and access our Cloud SQL database instance(s) we will leverage the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector).

The Cloud SQL Python Connector is a library that can be used alongside a database driver to allow users to easily connect to a Cloud SQL database without having to manually allowlist IP or manage SSL certificates. 🥳 🎉 🤩

### ♥️ Benefits of Using a Connector
Using a Cloud SQL connector provides the following benefits:

- 🔑 **IAM Authorization**: uses IAM permissions to control who/what can connect to your Cloud SQL instances.
- 🔒 **Improved Security**: uses robust, updated TLS 1.3 encryption and identity verification between the client connector and the server-side proxy, independent of the database protocol.
- 👍 **Convenience**: removes the requirement to use and distribute SSL certificates, as well as manage firewalls or source/destination IP addresses.
- 🪪 **IAM DB Authentication** (optional): provides support for Cloud SQL’s automatic IAM DB AuthN feature.

### 📱 Supported Dialects/Drivers
Google Cloud SQL and the Python Connector currently support the following dialects of SQL: **MySQL**, **PostgreSQL**, and **SQL Server**.

Depending on which dialect you are using for your relational database(s) the Python Connector will utilize a different database driver.

SUPPORTED DRIVERS:

*   **pymysql** (MySQL) 🐬
*   **pg8000** (PostgreSQL) 🐘
*   **pytds** (SQL Server) 🗄

Therefore, depending on the dialect of your database you will need to switch to the corresponding notebook!

📗 [**MySQL Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb)

📘 [**PostgreSQL Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/postgres_python_connector.ipynb) (this notebook)

📕 [**SQL Server Notebook**](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/sqlserver_python_connector.ipynb)

## 🚧 **Getting Started**
This notebook requires the following steps to be completed in order to successfully make Cloud SQL connections with the Cloud SQL Python Connector.

### 🔐 Authenticate to Google Cloud within Colab
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [1]:
from google.colab import auth

auth.authenticate_user()

### 🔗 Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab. 🏅 😀

In [4]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "vision-447321" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


### ☁ Configure Your Google Cloud Project
Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the
[Cloud SQL Client][client-role] role.

> 🚨 The user logged into this notebook will be used as the IAM principal and will be granted the Cloud SQL Client role.

[client-role]: https://cloud.google.com/sql/docs/mysql/roles-and-permissions

In [5]:
# grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [vision-447321].
bindings:
- members:
  - serviceAccount:service-706655025204@gcp-gae-service.iam.gserviceaccount.com
  role: roles/appengine.serviceAgent
- members:
  - serviceAccount:706655025204@cloudbuild.gserviceaccount.com
  role: roles/artifactregistry.reader
- members:
  - serviceAccount:service-706655025204@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:706655025204@cloudbuild.gserviceaccount.com
  role: roles/artifactregistry.writer
- members:
  - serviceAccount:706655025204@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-706655025204@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:vision-447321@appspot.gserviceaccount.com
  - user:SpenceReam@gmail.com
  role: roles/cloudsql.client
- members:
  - serviceAccount:vision-447321@appspot.gserviceaccount.co

2. Enable the [Cloud SQL Admin API][admin-api] within your project.

[admin-api]: https://console.cloud.google.com/apis/api/sqladmin.googleapis.com

In [6]:
# enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

## ☁️ Setting up Cloud SQL
A **Postgres** Cloud SQL instance is required for the following stages of this notebook.

###💽 **Create a Postgres Instance**
Running the below cell will verify the existence of a Cloud SQL instance or create a new one if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_name = "vision-db" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for 'postgres' database user: ")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_14 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On

Creating new Cloud SQL instance...


### 🥪 Create a Sandwiches Database
A `sandwiches` database will be used in later steps when connecting to and querying a Cloud SQL database.

To create a `sandwiches` database within your Cloud SQL instance run the below command:

In [8]:
!gcloud sql databases create sandwiches --instance={instance_name}

Created database [sandwiches].
instance: vision-db
name: sandwiches
project: vision-447321


### 🧑‍🍳 Create a Chef Database User
To create the `chef` database user that is used throughout the notebook, run the following `gcloud` command.

In [9]:
!gcloud sql users create chef \
  --instance={instance_name} \
  --password="food"

Created user [chef].


<img src='https://i.pinimg.com/originals/12/64/dd/1264dd5ff31fbc65c5edbb5e1a71830e.gif' class="center"/>


## 🐍 Python Connector Usage
Let's now connect to Cloud SQL using the Python Connector! 🚀 ⭐ 🐍

### 🎟 **Configuring Credentials**
The Cloud SQL Python Connector uses [**Application Default Credentials (ADC)**](https://cloud.google.com/docs/authentication) strategy for resolving credentials.

> 💡 Using the Python Connector in Cloud Run, App Engine, or Cloud Functions will automatically use the service account deployed with each service, allowing this step to be skipped. ✅

Please see the [google.auth](https://google-auth.readthedocs.io/en/master/reference/google.auth.html) package documentation for more information on how these credentials are sourced.

This means setting default credentials was previously done for you when you ran:
```python
from google.colab import auth

auth.authenticate_user()
```


### 💻 **Install Code Dependencies**
It is recommended to use the Connector alongside a library that can create connection pools, such as [SQLAlchemy](https://www.sqlalchemy.org/).
This will allow for connections to remain open and be reused, reducing connection overhead and the number of connections needed

Let's `pip install` the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector) as well as [SQLAlchemy](https://www.sqlalchemy.org/), using the below command.

In [10]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

## 🐘 **Connect to a Postgres Instance**
We are now ready to connect to a Postgres instance using the Cloud SQL Python Connector! 🐍 ⭐ ☁


Let's set some parameters that are needed to connect properly to a Cloud SQL instance:
*   `INSTANCE_CONNECTION_NAME` : The connection name to your Cloud SQL Instance, takes the form `PROJECT_ID:REGION:INSTANCE_NAME`.
*   `DB_USER` : The user that the connector will use to connect to the database.
*   `DB_PASS` : The password of the DB_USER.
*   `DB_NAME` : The name of the database on the Cloud SQL instance to connect to.

In [11]:
# initialize parameters
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "chef"
DB_PASS = "food"
DB_NAME = "sandwiches"

Your instance connection name is: vision-447321:us-central1:vision-db


### ✅ **Basic Usage**
To connect to Cloud SQL using the connector, initialize a `Connector` object and call its `connect` method with the proper input parameters.

The `connect` method takes in the parameters we previously defined, as well as a few additional parameters such as:
*   `driver`: The name of the database driver to connect with.
*   `ip_type` (optional): The IP type (public or private)  used to connect. IP types can be either `IPTypes.PUBLIC` or `IPTypes.PRIVATE`. ([Example](#scrollTo=yjAPpIDdRfu2))
*   `enable_iam_auth`: (optional) Boolean enabling IAM based authentication. ([Example](#scrollTo=GpVKrv0TCXje))

Let's show an example! 🤘 🙌

In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

To use this connector with SQLAlchemy, we use the `creator` argument for `sqlalchemy.create_engine`

Now that we have established a connection pool, let's write a query! 🎉 📝

In [ ]:
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  db_conn.execute(
    sqlalchemy.text(
      "CREATE TABLE IF NOT EXISTS ratings "
      "( id SERIAL NOT NULL, name VARCHAR(255) NOT NULL, "
      "origin VARCHAR(255) NOT NULL, rating FLOAT NOT NULL, "
      "PRIMARY KEY (id));"
    )
  )

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # insert data into our ratings table
  insert_stmt = sqlalchemy.text(
      "INSERT INTO ratings (name, origin, rating) VALUES (:name, :origin, :rating)",
  )

  # insert entries into table
  db_conn.execute(insert_stmt, parameters={"name": "HOTDOG", "origin": "Germany", "rating": 7.5})
  db_conn.execute(insert_stmt, parameters={"name": "BÀNH MÌ", "origin": "Vietnam", "rating": 9.1})
  db_conn.execute(insert_stmt, parameters={"name": "CROQUE MADAME", "origin": "France", "rating": 8.3})

  # commit transactions
  db_conn.commit()

  # query and fetch ratings table
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

  # show results
  for row in results:
    print(row)

You have successfully been able to connect to a Cloud SQL instance from this notebook and make a query. YOU DID IT! 🕺 🎊 💃

<img src=https://media.giphy.com/media/MtHGs1yo4FFKrIs55L/giphy.gif />

To close the `Connector` object's background resources, call it's `close() ` method at the end of your code as follows:


In [ ]:
# cleanup connector object
connector.close()

### 🪪  IAM Database Authentication
[Automatic IAM database authentication](https://cloud.google.com/sql/docs/postgres/authentication#automatic) is supported for **Postgres** Cloud SQL instances.

> 💡 This allows an IAM user to establish an authenticated connection to a Postgres database without having to set a password and enabling the `enable_iam_auth` parameter in the connector's `connect` method.

> 🚨 If you are using a pre-existing Cloud SQL instance within this notebook you may need to [configure Cloud SQL instance to allow IAM authentication](https://cloud.google.com/sql/docs/postgres/create-edit-iam-instances#configuring_existing_instances_for) by setting the `cloudsql.iam_authentication` database flag to `On`.
(Cloud SQL instances created within this notebook already have it enabled)


IAM principals wanting to use IAM authentication to connect to a Cloud SQL instance require the `Cloud SQL Instance User` and `Cloud SQL Client` IAM role.

Let's add the Cloud SQL Instance User role to the IAM account logged into this notebook. (Client role previously granted)

In [ ]:
# add Cloud SQL Instance User role to current logged in IAM user
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.instanceUser"

Now the current IAM user can be added to the Cloud SQL instance as an IAM database user.

In [ ]:
# add current logged in IAM user to database
!gcloud sql users create {current_user[0]} \
  --instance={instance_name} \
  --type=cloud_iam_user

Finally, let's update our `getconn` function to connect to our Cloud SQL instance with IAM database authentication enabled.

> ⚠️ The below sample is a limited example as it logs in to the Cloud SQL instance and outputs the current time. By default new IAM database users have no permissions on a Cloud SQL instance. To connect to specific tables and perform more complex queries, permissions must be granted at the database level. ([Grant Database Privileges to the IAM user](https://cloud.google.com/sql/docs/postgres/add-manage-iam-users#grant-db-privileges))

In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# IAM database user parameter (IAM user's email)
IAM_USER = current_user[0]

# initialize connector
connector = Connector()

# getconn now using IAM user and requiring no password with IAM Auth enabled
def getconn():
    conn = connector.connect(
      INSTANCE_CONNECTION_NAME,
      "pg8000",
      user=IAM_USER,
      db="postgres",
      enable_iam_auth=True
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

# connect to connection pool
with pool.connect() as db_conn:
    # get current datetime from database
    results = db_conn.execute(sqlalchemy.text("SELECT NOW()")).fetchone()

    # output time
    print("Current time: ", results[0])

# cleanup connector
connector.close()

Sucess! You were able to connect to Cloud SQL as an IAM authenticated user using the Cloud SQL Python Connector! 🍾 👏 🏆

<img src="https://media.giphy.com/media/YTbZzCkRQCEJa/giphy.gif" />

## 🗑 Clean Up Notebook Resources
Make sure to delete your Cloud SQL instance when your are finished with this notebook to avoid further costs. 💸 💰

In [ ]:
# delete Cloud SQL instance
!gcloud sql instances delete {instance_name}

## ✍ **Appendix**
Additional information provided for connecting to a Cloud SQL instance using private IP connections.


### 🔒 **Using Private IP Connections**
By default the connector connects to the Cloud SQL instance database using a Public IP address.

**Private IP** connections are also supported by the connector and can be easily enabled through the `ip_type` parameter in the connector's `connect` method.

> ⚠️ To connect via Private IP, the Cloud SQL instance being connected to must have a Private IP address configured within a VPC Network. ([How to Configure Private IP](https://cloud.google.com/sql/docs/postgres/configure-private-ip))

> 🚫 The below cell is a working sample but will not work within this notebook as the notebook is not within your VPC Network! The cell should be copied into an environment (Cloud Run, Cloud Functions, App Engine etc.) that has access to the VPC Network.

 > [Connecting Cloud Run to a VPC Network](https://cloud.google.com/run/docs/configuring/connecting-vpc)

Let's update our `getconn` function to connect to our Cloud SQL instance with Private IP.

In [ ]:
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy

# initialize connector
connector = Connector()

# getconn now set to private IP
def getconn():
    conn = connector.connect(
      INSTANCE_CONNECTION_NAME, # <PROJECT-ID>:<REGION>:<INSTANCE-NAME>
      "pg8000",
      user=DB_USER,
      password=DB_PASS,
      db=DB_NAME,
      ip_type=IPTypes.PRIVATE
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

# connect to connection pool
with pool.connect() as db_conn:
    # query database and fetch results
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

    # show results
    for row in results:
        print(row)

# cleanup connector
connector.close()